In [1]:
import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
# import pctils as pc

import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
# import pctils as pc
# SKLearn related imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
import spacy
import hashlib

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin


In [2]:
dummy = pd.read_csv('data/book_review_labelled_data.csv')[0:3]
dummy.head()

,reviewerID,reviewerName,reviewText,overall,summary,reviewTime,rates_count,helpful_count,rating
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...",4,"Quite readable, nicely done","12 6, 2001",40,37,4
1,A1XTKTLNSCRLDS,Ellen Rappaport,Detective Inspector Erlendur Sveinsson is at h...,5,Mesmerizing in depth,"02 23, 2014",0,0,5
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,3,10-minute life lessons for kids,"02 12, 2013",3,0,3


In [3]:
text_docs = dummy.reviewText
text_docs[0]
tk = WordPunctTokenizer() 
def tkze(tk,text_docs): 
    return [" ".join(tk.tokenize(doc)) for doc in text_docs]

docs = tkze(tk,text_docs)

In [4]:
# number of words 
def get_nb_words(docs): 
    return ([len(doc.split()) for doc in docs])

# get_nb_words(docs)

In [5]:
# average word size 
def get_av_word_size(docs):
    av_words_size_per_doc = [] 
    for doc in docs: 
        words_size = [len(word) for word in doc.split()]
        av_size = np.mean(np.array(words_size))
        av_words_size_per_doc.append(av_size)
    return av_words_size_per_doc
# get_av_word_size(docs)  

In [24]:
# number of unique words (proxy for how rich is the reviewer vocabulary)
# not really sure if this feature will be useful (maybe couple with PoS) 
def get_nb_unique_words(docs):
    unique_words = [len(set(word for word in doc)) for doc in docs] 
    normalized = (np.array(unique_words)/np.array(get_nb_words(docs))).tolist()
    return normalized
# get_nb_unique_words(docs)


[0.1652892561983471, 0.15625, 0.8571428571428571]

In [7]:
# important for spacy related functions
# nlp = spacy.load('en_core_web_sm')
# nlp_docs = list(tqdm(nlp.pipe(docs), total=len(docs)))

In [8]:
# getting NERs (proxy for attention to detail by reviewers that describe book parts, e.g. naming book charters)
def get_most_common_persons(nlp_docs, n_most_common=10): 
    matcher = Matcher(nlp.vocab)
    pattern = [[{'ENT_TYPE':'PERSON'}]]
    matcher.add('ppl', pattern)

    persons = []
    for doc in nlp_docs: 
        matches = matcher(doc)
        for match_id, start, end in matches: 
            persons.append(str(doc[start:end]))

    most_common_ents = [person for person in Counter(persons).most_common()[:n_most_common]]
    return most_common_ents

# get_most_common_persons(nlp_docs)    

In [31]:
# number of predefined parts of speech 
def get_nb_PoS(nlp_docs): # docs tokenized string  
    PoS_list = ['ADJ','NOUN','VERB'] 
    nb_PoS = [] 
    for doc in nlp_docs: 
        total_PoS = [token.pos_ for token in doc if token.pos_ in PoS_list]  
        nb_in_doc = sum([Counter(total_PoS)[x] for x in PoS_list])
        nb_PoS.append(nb_in_doc)
    return nb_PoS

# get_nb_PoS(nlp_docs)

[115, 98, 11]

In [27]:
def get_nb_unique_PoS(nlp_docs): # docs tokenized string  
    PoS_list = ['ADJ','NOUN'] 
    nb_unique_PoS = [] 
    nb_PoS = []
    word4PoS = [] 
    for doc in nlp_docs: 
        total_PoS = [token.pos_ for token in doc]
        mask = np.array([True if PoS in PoS_list else False for PoS in total_PoS])
        # print(mask[:10])
        words = np.array([token.text for token in doc])
        words_under_selected_PoS = words[mask].tolist()
        word4PoS.append(words_under_selected_PoS)
        # print(words_under_selected_PoS)
        unique_words = get_nb_unique_words(words_under_selected_PoS)
        nb_unique_PoS.append(len(unique_words))
        nb_PoS.append(len(words_under_selected_PoS))
        
    ratio = np.array(nb_unique_PoS)/ np.array(nb_PoS)
    return ratio.tolist(), word4PoS
    
# get_nb_unique_PoS(nlp_docs)

In [28]:
def add_extra_features(df, docs, nlp_docs): 
    df["nb_words"] = pd.Series(get_nb_words(docs)) 
    df["av_word_size"] = pd.Series(get_av_word_size(docs))
    df["nb_unique_words"] = pd.Series(get_nb_unique_words(docs)) # not work properly 
    
    
    # get_most_common_persons(nlp_docs, n_most_common=10) 
    
    df["nb_PoS"] = pd.Series(get_nb_PoS(nlp_docs))
    df["nb_unique_PoS"] = pd.Series(get_nb_unique_PoS(nlp_docs)[0])
    return df

In [29]:
df = dummy.copy() 
nlp = spacy.load('en_core_web_sm')
nlp_docs = list(tqdm(nlp.pipe(docs), total=len(docs)))
add_extra_features(df, docs, nlp_docs)

100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 35.26it/s]


,reviewerID,reviewerName,reviewText,overall,summary,reviewTime,rates_count,helpful_count,rating,nb_words,av_word_size,nb_unique_words,nb_PoS,nb_unique_PoS
0,A3UPFTGAWZ3G2R,David J. Loftus,"Jenkins, a history professor and Member of Par...",4,"Quite readable, nicely done","12 6, 2001",40,37,4,363,4.223140,0.165289,115,1.0
1,A1XTKTLNSCRLDS,Ellen Rappaport,Detective Inspector Erlendur Sveinsson is at h...,5,Mesmerizing in depth,"02 23, 2014",0,0,5,288,4.399306,0.156250,98,1.0
2,A1A77B6DQQH436,"crescamp ""esc""",I didn't read this. I purchased it for a gift...,3,10-minute life lessons for kids,"02 12, 2013",3,0,3,28,3.285714,0.857143,11,1.0


In [30]:
!explorer.exe .
